In [1]:
!pip install transformers accelerate bitsandbytes sentence-transformers wikipedia duckduckgo-search


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 109.0 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=4eecc33bfd2fd4089d532157999b485d397ba51cdd3ab964a7188faee746171a
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [8]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate


In [9]:
pip install -U bitsandbytes

In [10]:
import pandas as pd
import json
import re
from tqdm import tqdm  # optional for progress bar

# Load your dataset
df = pd.read_csv("dataset_final1.csv")  # replace with your CSV path


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# LLM pipeline
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.2
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
def analyze_article_json(article):
    import json, re
    """
    Analyzes article bias, credibility, and emotional manipulation using Mistral.
    Returns parsed JSON or error.
    """
    prompt = f"""
    You are a media analysis expert.
    Analyze the following news article and output ONLY valid JSON with real values.

    Fields:
    - credibility_score: integer (0-100)
    - Political bias: "Left", "Right", "Center", or "Unknown"
    - emotional_tone: "Neutral", "Emotional", or "Manipulative"
    - persuasive_techniques: list of manipulative techniques (if any)
    - bias_summary: short summary of bias
    - reasoning: short reasoning

    Article:
    {article}

    Output ONLY JSON (no explanations, no markdown).
    """

    result = llm(prompt)[0]["generated_text"]

    # Extract JSON
    match = re.search(r"\{.*\}", result, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except Exception as e:
            return {"error": f"JSON parse error: {e}", "raw_output": result}
    else:
        return {"error": "No JSON found", "raw_output": result}


In [11]:
def llm_decide_bias(bias_json):
    import json, re

    prompt = f"""
You are a bias evaluation expert.
You will receive a JSON analysis of a news article from another model.

Decide whether the article is "Biased" or "Not Biased" based on the data.

Rules:
- "Biased" if:
    • political_bias is not "Center" and not "Unknown"
    • emotional_tone is "Emotional" or "Manipulative"
    • persuasive_techniques list is not empty
    • credibility_score < 60
- Otherwise: "Not Biased"

Return ONLY valid JSON in this exact format:
{{
  "bias_decision": "Biased" or "Not Biased",
  "reasoning": "short justification"
}}

Input JSON:
{json.dumps(bias_json, indent=2)}

Output ONLY JSON — no explanations, no markdown, no ``` backticks.
"""

    result = llm(prompt)[0]["generated_text"]

    # Extract the last JSON object
    matches = re.findall(r"\{[\s\S]*?\}", result)
    if matches:
        json_text = matches[-1]
        try:
            return json.loads(json_text)
        except Exception as e:
            return {"error": f"JSON parse error: {e}", "raw_output": result}
    else:
        return {"error": "No JSON found", "raw_output": result}


In [12]:
# Initialize empty lists to store results
credibility_scores = []
political_biases = []
emotional_tones = []
persuasive_techniques_list = []
bias_summaries = []
reasonings = []
bias_decisions = []
decision_reasonings = []

In [ ]:
# Iterate over articles
for idx, row in tqdm(df.iterrows(), total=len(df)):
    article_content = row["content"]

    # Step 1: Analyze the article
    bias_json = analyze_article_json(article_content)

    # Handle cases where JSON failed
    if "error" in bias_json:
        credibility_scores.append(None)
        political_biases.append(None)
        emotional_tones.append(None)
        persuasive_techniques_list.append(None)
        bias_summaries.append(None)
        reasonings.append(None)
        bias_decisions.append(None)
        decision_reasonings.append(None)
        continue

    # Save Step 1 results
    credibility_scores.append(bias_json.get("credibility_score"))
    political_biases.append(bias_json.get("Political bias"))
    emotional_tones.append(bias_json.get("emotional_tone"))
    persuasive_techniques_list.append(bias_json.get("persuasive_techniques"))
    bias_summaries.append(bias_json.get("bias_summary"))
    reasonings.append(bias_json.get("reasoning"))

    # Step 2: Decide bias
    bias_decision_json = llm_decide_bias(bias_json)

    if "error" in bias_decision_json:
        bias_decisions.append(None)
        decision_reasonings.append(None)
    else:
        bias_decisions.append(bias_decision_json.get("bias_decision"))
        decision_reasonings.append(bias_decision_json.get("reasoning"))

  0%|          | 0/4282 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 1/4282 [00:38<46:20:16, 38.97s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 2/4282 [01:51<69:54:29, 58.80s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 3/4282 [02:08<47:10:58, 39.70s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 4/4282 [02:29<38:23:22, 32.31s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 5/4282 [02:56<36:11:15, 30.46s/it]You seem to be using the 

In [ ]:
# Add new columns to dataframe
df["credibility_score"] = credibility_scores
df["political_bias"] = political_biases
df["emotional_tone"] = emotional_tones
df["persuasive_techniques"] = persuasive_techniques_list
df["bias_summary"] = bias_summaries
df["analysis_reasoning"] = reasonings
df["bias_decision"] = bias_decisions
df["decision_reasoning"] = decision_reasonings

# Save updated dataframe
df.to_csv("articles_with_bias_analysis.csv", index=False, encoding="utf-8-sig")
print("✅ Dataset updated and saved with bias analysis results.")